In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
import lightgbm as lgb
from multiprocessing import Pool        
warnings.filterwarnings('ignore')

fea_pkl_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/pkl/fea_pkl/'
model_pkl_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/pkl/model_pkl/'
Metric_pkl_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/pkl/Metric_pkl/'
datasets_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/datasets/'
fea_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Features/'
pred_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Predict/'
sub_path = '/content/drive/My Drive/m5-forecasting-accuracy/SilverCode(final)/Predict/sub/'


BASE = fea_pkl_path + 'grid_part_1.pkl'
PRICE = fea_pkl_path + 'grid_part_2.pkl'
CALENDAR = fea_pkl_path + 'grid_part_3.pkl'
LAGS = fea_pkl_path + 'grid_part_4.pkl'
MEAN_ENC = fea_pkl_path + 'grid_part_5.pkl'

In [ ]:
## 设定随机种子
def seed_everything(seed=0):
  random.seed(seed)
  np.random.seed(seed)

## 按照商店读取数据
def get_data_by_store(store):
    # 读取和连接基本特征
    df = pd.concat([pd.read_pickle(BASE), # 9
             pd.read_pickle(PRICE).iloc[:,2:], # 11
             pd.read_pickle(CALENDAR).iloc[:,2:], # 15 
             pd.read_pickle(LAGS).iloc[:,3:], # 37
             pd.read_pickle(MEAN_ENC)[mean_features]], # 6  
             axis=1) 
    # df：(47735397, 78)由于行数相同，可以横向拼接
    df = df[df['store_id'] == store] # df['store_id'] == store 返回布尔Series，df[df['store_id'] == store] 然后取True的行构成的dataframe
    features = [col for col in list(df) if col not in remove_features] # 78 - 7 = 71
    df = df[['id', 'd', TARGET] + features] # 71 + 3 = 74 相当于去掉了 'state_id', 'store_id', 'release', 'Holiday'(在CALENDAR里面)
    # 选训练开始的天数
    df = df[df['d'] >= START_TRAIN].reset_index(drop=True) # drop = True去掉生成的原索引列
    return df, features


## 读取测试数据
def get_base_test():
  base_test = pd.DataFrame()

  if USE_AUX:
    model_path = model_pkl_path
  else:
    model_path=''

  for store_id in STORES_IDS: 
    temp_df = pd.read_pickle(model_path + 'xgb_test_{}_B.pkl'.format(store_id)) # 不含lag + rolling特征
    temp_df['store_id'] = store_id
    base_test = pd.concat([base_test, temp_df]).reset_index(drop=True) # 纵向拼接
  
  return base_test
## 制作lag特征
def make_lag(LAG_DAY):
  lag_df = base_test[['id','d',TARGET]]
  col_name = 'sales_lag_' + str(LAG_DAY)
  lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
  return lag_df[[col_name]]
## 递归特征
def make_lag_roll(LAG_DAY):
  shift_day = LAG_DAY[0]
  roll_wind = LAG_DAY[1]
  lag_df = base_test[['id','d',TARGET]] 
  col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
  lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean()) 

  return lag_df[[col_name]]
## 多线程执行，用于测试集融合
def df_parallelize_run(func, t_split):
  num_cores = np.min([N_CORES,len(t_split)])
  pool = Pool(num_cores)
  df = pd.concat(pool.map(func, t_split), axis=1)
  pool.close()
  pool.join()
  return df

# lag + rolling
SHIFT_DAY = 28
N_LAGS = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY, SHIFT_DAY + N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
  for j in [7,14,30,60]:
    ROLS_SPLIT.append([i,j])

# grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)


VER = 1 # 设置模型的版本
SEED = 42 
seed_everything(SEED) # 消除随机性
N_CORES = psutil.cpu_count() # 可使用的CPU内核

TARGET = 'sales' # Label
# START_TRAIN = 1600 # 快速训练，验证是否有BUG
START_TRAIN = 0 # 真正训练
END_TRAIN = 1941
P_HORIZON = 28 # 预测范围
USE_AUX = True # 使用预训练好的模型

remove_features = ['id', 'state_id', 'store_id', 'release', 'Holiday', 'd', TARGET]          
mean_features = ['enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std'] 
# 按商店分别训练，每个商店可以只能按类别、部门、商品的销量聚合取mean\std，故只选这6个特征
STORES_IDS = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']

## 每个商店的xgb参数及类别特征处理

In [ ]:
xgb_params_common = {
            'booster': 'gbtree',
            'objective': 'reg:tweedie',
            'eval_metric': 'rmse',
            'min_data_in_leaf': 11801,
            'lambda': 0.021982796763644744, 
            'eta': 0.04723157294394453,
            'colsample_bytree': 0.9329928507911868, 
            'subsample': 0.5951931300022044,    
            'max_bin':67,
            'num_leaves': 25631,
            'sub_row': 0.6184421797679618,
            'subsample_freq': 3,
            'silent':1, 
            'seed': 42,
            'tree_method': 'hist',
            'tweedie_variance_power':1.1833186379351004,
            'boost_from_average': False
           }

In [ ]:
# 下面处理类别特征需要用到的工具包
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn import preprocessing
from keras.layers.embeddings import Embedding
from keras.models import Sequential
import tensorflow as tf
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

Using TensorFlow backend.


## 每个商店单独训练模型

In [ ]:
############################################# Train Models ###############################################
for store_id in STORES_IDS: # 每个商店单独训练
    print('Train', store_id)
    grid_df, features_columns = get_data_by_store(store_id) 
    calendar_win = pd.read_csv(fea_path + 'CA1_TX2_TX3_holidays.csv')
    calendar_win['d'] = calendar_win['d'].apply(lambda x: x.split('_',2)[1]).astype(np.int16) 
    calendar_win['event_name_1_win'] = calendar_win['event_name_1_win'].astype('category') 
    grid_df = grid_df.merge(calendar_win, on='d', how='left') 

    #################################### PCA ####################################
    pca_fea = ['snap_CA', 'snap_TX', 'snap_WI'] # 这三个分别onehot编码之后，再PCA，能保持100%的信息
    pca_df = pd.DataFrame()
    pca = PCA(n_components = 'mle')
    OneHot_snap = pd.get_dummies(grid_df[pca_fea])
    pca_df = pd.concat([pd.DataFrame(pca.fit_transform(OneHot_snap.iloc[:,0:2]), columns = ['snap_CA_pca']),
               pd.DataFrame(pca.fit_transform(OneHot_snap.iloc[:,2:4]), columns = ['snap_TX_pca']),
               pd.DataFrame(pca.fit_transform(OneHot_snap.iloc[:,4:6]), columns = ['snap_WI_pca'])], axis=1)
    #################################### One-Hot ####################################
    one_hot_fea = ['cat_id', 'event_type_2'] # 'cat_id'特征值个数为3，'event_type_2'：2
    one_hot_df = pd.get_dummies(grid_df[one_hot_fea])  
    ############################ LabelEncoder + Enbedding ##########################
    ## 先用label编码 
    embedding_fea = ['dept_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_name_1_win']
    for i in embedding_fea:
      if grid_df[i].dtypes=='category':
        le = preprocessing.LabelEncoder()
        grid_df[i] = le.fit_transform(list(grid_df[i].values))

    intput_vec = [7,31,5,5,32]
    output_vec = [3,10,3,3,10]
    embedding_df = pd.DataFrame()
    int_j = 0
    out_k = 0
    ## 再用embedding进行特征降维
    for i in embedding_fea:
      model = Sequential()
      model.add(Embedding(intput_vec[int_j], output_vec[out_k], input_length = 1))
      model.compile(loss='mean_squared_error', optimizer='rmsprop')
      input_array = grid_df[[i]].values
      output_array = model.predict(input_array).reshape((-1, output_vec[out_k]))
      # intput_vec[int_j]：不同特征值的个数(词汇表大小) output_vec[out_k]：词向量的维度
      # 输入尺寸为 (batch_size, sequence_length) 的 2D 张量
      # 输出尺寸为 (batch_size, sequence_length, output_dim) 的 3D 张量，比如：'dept_id' embedding之后(4873639, 1, 3)，需要reshape一下。
      temp_df = pd.DataFrame(output_array)
      embedding_df = pd.concat([embedding_df, temp_df], axis = 1)
      int_j += 1
      out_k += 1

    embedding_df.columns = [f'eme_{col}' for col in range(len(embedding_df.columns))]

    category_feature = ['item_id', 'dept_id', 'cat_id', 'snap_CA', 'snap_TX', 'snap_WI', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'event_name_1_win'] 
                # 由于item_id有3049个特征值，没办法处理，剔除掉，其他的分开处理
    grid_df.drop(columns = category_feature, inplace = True) ## BUG：不能剔除掉 "id" 否则后面没办法迭代按照 id 生成迭代预测的特征了。
    ## 融合到一起 
    xgb_df = pd.concat([embedding_df, one_hot_df, pca_df], axis=1)
    grid_df = pd.concat([grid_df, xgb_df], axis = 1)
    features_columns = list(set(features_columns).difference(set(category_feature))) + xgb_df.columns.to_list()
    # set(listA).difference(set(listB)) 去掉AB的交集，取A剩下的。set运算完之后，返回的还是set，需要转化为list
    del xgb_df, embedding_df, one_hot_df, pca_df
    print('finish：{}'.format(store_id))

    xgb_params = xgb_params_common

    train_mask = grid_df['d'] <= END_TRAIN - P_HORIZON # 1<= train <=1941-28 
    valid_mask = (grid_df['d'] > (END_TRAIN - P_HORIZON)) & (grid_df['d'] <= END_TRAIN) # 1941-28< valid <=1941
    preds_mask = grid_df['d'] > (END_TRAIN - 100) 

    train_data = xgb.DMatrix(grid_df[train_mask][features_columns], label = grid_df[train_mask][TARGET])
    valid_data = xgb.DMatrix(grid_df[valid_mask][features_columns], label = grid_df[valid_mask][TARGET])

    # 构造一个后面预测用的数据集 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col] # '_tmp_' 滞后 + rolling 比如：rolling_mean_tmp_14_60 去掉了12列
    grid_df = grid_df[keep_cols] # CA1：75 - 12 = 63 (390272, 63)  [1842, 1969] 128天
    grid_df.to_pickle(model_pkl_path + 'xgb_test_{}_B.pkl'.format(store_id))
    
    # 训练模型
    seed_everything(SEED)
    watchlist = [(valid_data, 'eval')]
    estimator = xgb.train(xgb_params, train_data, evals = watchlist, num_boost_round = 20000, verbose_eval = 100, early_stopping_rounds = 100)
    # early_stopping_rounds 验证集的误差迭代到一定程度在100次内不能再继续降低，就停止迭代。要求evals里至少有一个元素。
    # 存储模型为二进制文件，下次读取的时候，速度更快
    model_name = model_pkl_path + 'xgb_model_{}_v{}_B.bin'.format(store_id, str(VER))
    pickle.dump(estimator, open(model_name, 'wb'))

    del grid_df, train_data, valid_data, estimator
    gc.collect()

## 遇到的问题：训练的时候只迭代几步就结束了
# Train CA_1
# finish：CA_1
# [0]	train-rmse:4.58096	eval-rmse:3.72794
# [9]	train-rmse:4.2113	eval-rmse:3.37202
# Train CA_2
## 解决办法：在train中设置：num_boost_round，因为没有设置迭代次数的话，默认只迭代10次就停止了。 
## Xgboost中的迭代次数 num_boost_round 和LightGBM中的迭代次数 n_estimators 是一样的

Train CA_1
finish：CA_1
[0]	eval-rmse:3.72797
Will train until eval-rmse hasn't improved in 100 rounds.
[100]	eval-rmse:2.02326
[200]	eval-rmse:2.01679
[300]	eval-rmse:2.01333
[400]	eval-rmse:2.01175
[500]	eval-rmse:2.00901
[600]	eval-rmse:2.00694
[700]	eval-rmse:2.00617
[800]	eval-rmse:2.00645
Stopping. Best iteration:
[787]	eval-rmse:2.0044

Train CA_2
finish：CA_2
[0]	eval-rmse:3.30137
Will train until eval-rmse hasn't improved in 100 rounds.
[100]	eval-rmse:1.93221
[200]	eval-rmse:1.91596
[300]	eval-rmse:1.90753
[400]	eval-rmse:1.90361
[500]	eval-rmse:1.89982
[600]	eval-rmse:1.89868
[700]	eval-rmse:1.89746
[800]	eval-rmse:1.89663
[900]	eval-rmse:1.89545
[1000]	eval-rmse:1.89421
[1100]	eval-rmse:1.89342
Stopping. Best iteration:
[1094]	eval-rmse:1.89332

Train CA_3
finish：CA_3
[0]	eval-rmse:5.02822
Will train until eval-rmse hasn't improved in 100 rounds.
[100]	eval-rmse:2.41105
Stopping. Best iteration:
[82]	eval-rmse:2.40769

Train CA_4
finish：CA_4
[0]	eval-rmse:1.98322
Will train u

In [ ]:
MODEL_FEATURES = features_columns

## 预测

In [ ]:
USE_AUX = True
all_preds = pd.DataFrame()
base_test = get_base_test() 
# 3902720 rows × 66 columns 62 + 4（event_name_1_win、store_id、event_name_2_win、event_name_1_win_1）
# 记住：后面这个特征是新加的，原来只有62 + 12(lag+rolling) = 72
main_time = time.time()

for PREDICT_DAY in range(1,29):    
  print('Predict | Day:', PREDICT_DAY)
  start_time = time.time() 
  grid_df = base_test.copy() 
  grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1) 
  # print(grid_df.columns)
  print('finish roll')
  # 3902720(30490×128) × 78(66 + 12(lag+rolling)) 
  # rolling_mean_tmp_14_60 每组128个数据，相当于每个id的128天这个时序，lag = 14，到1942到1942+14天有数据
  # 具体看一下 datasets 文件夹里面的 HOUSEHOLD_2_490_WI_3.csv文件 
  for store_id in STORES_IDS:
    model_bin = 'xgb_model_{}_v{}_B.bin'.format(store_id, str(VER))
    if USE_AUX:
      model_path = model_pkl_path + model_bin
    estimator = pickle.load(open(model_path, 'rb')) # 读取训练好的模型


    day_mask = base_test['d'] == (END_TRAIN + PREDICT_DAY) # 1942、1943、...、1969
    store_mask = base_test['store_id'] == store_id
    mask = (day_mask) & (store_mask)

    test = xgb.DMatrix(grid_df[mask][MODEL_FEATURES]) 
    base_test[TARGET][mask] = estimator.predict(test) # 测试集也要封装成DMatrix形式

  temp_df = base_test[day_mask][['id',TARGET]]
  # print(temp_df)
  temp_df.columns = ['id', 'F' + str(PREDICT_DAY)]
  if 'id' in list(all_preds):
    all_preds = all_preds.merge(temp_df, on=['id'], how='left')
  else:
    all_preds = temp_df.copy()
      
  print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
          ' %0.2f min total |' % ((time.time() - main_time) / 60),
          ' %0.2f day sales |' % (temp_df['F' + str(PREDICT_DAY)].sum()))
  del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds.to_csv(pred_path + 'per_store_pred(xgb)_B.csv',index=False)


# 用于生成提交预测的文件
submission = pd.read_csv(datasets_path + 'sample_submission.csv')[['id']] 
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv(sub_path + 'submission_by_stores(xgb)_B.csv', index=False) 

Predict | Day: 1
Index(['id', 'd', 'sales', 'sell_price', 'price_max', 'price_min', 'price_std',
       'price_mean', 'price_norm', 'price_nunique',
       ...
       'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60',
       'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14',
       'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60',
       'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14',
       'rolling_mean_tmp_14_30', 'rolling_mean_tmp_14_60'],
      dtype='object', length=102)
finish roll
##########  0.67 min round |  0.67 min total |  39163.18 day sales |
Predict | Day: 2
Index(['id', 'd', 'sales', 'sell_price', 'price_max', 'price_min', 'price_std',
       'price_mean', 'price_norm', 'price_nunique',
       ...
       'rolling_mean_tmp_1_30', 'rolling_mean_tmp_1_60',
       'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14',
       'rolling_mean_tmp_7_30', 'rolling_mean_tmp_7_60',
       'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14',
       'rolling_mean_tmp_14_30', 'rolling_mean_tm

In [ ]:
################################## 本地wrmse #############################################################
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import gc

# 加载前面预先计算好的各个权重
sw_df = pd.read_pickle(Metric_pkl_path + 'sw_df.pkl')
S = sw_df.s.values
W = sw_df.w.values
SW = sw_df.sw.values
roll_mat_df = pd.read_pickle(Metric_pkl_path + 'roll_mat_df.pkl')
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)


def rollup(v):
    return (v.T*roll_mat_csr.T).T

# 计算 WRMSSE 评估指标
def wrmsse(preds, y_true, score_only=False, s = S, w = W, sw=SW):
    '''
    preds - Predictions: pd.DataFrame of size (30490 rows, N day columns)
    y_true - True values: pd.DataFrame of size (30490 rows, N day columns)
    sequence_length - np.array of size (42840,)
    sales_weight - sales weights based on last 28 days: np.array (42840,)
    '''
    
    if score_only:
        return np.sum(
                np.sqrt(
                    np.mean(
                        np.square(rollup(preds.values-y_true.values))
                            ,axis=1)) * sw )*(1/12)
    else: 
        score_matrix = (np.square(rollup(preds.values-y_true.values)) * np.square(w)[:, None])  / s[:, None]
        score = np.sum(np.sqrt(np.mean(score_matrix,axis=1)))*(1/12)
        return score, score_matrix

In [ ]:
## 预测值
cols = [f'F{i}' for i in range(1,29)]
pred = all_preds1[cols]
## 真值1
cols_1 = [f'd_{i}' for i in range(1886, 1914)]
true_1 = pd.read_csv(datasets_path + 'sales_train_validation.csv')[col2]
## 真值2
cols_2 = [f'd_{i}' for i in range(1914, 1942)]
true_2 = pd.read_csv(datasets_path + 'sales_train_evaluation.csv')[cols_2]
## 进行测试
wrmsse(true_2, pred ,score_only=True)  

## 某个商店进行贝叶斯调参

In [ ]:
STORES_IDS = ['CA_1']
grid_df, features_columns = get_data_by_store('CA_1')
day = 1941

grid_df1 = grid_df[grid_df['d'] <= day]
del grid_df
!pip install bayesian-optimization

from bayes_opt import BayesianOptimization

def rmse(y, y_pred):
  return np.sqrt(np.mean(np.square(y - y_pred)))
df = grid_df1
fe = features_columns
# 1000到1941-28-28 作为训练集
tr_x, tr_y = df[(df['d'] >= 1000) & (df['d'] <= (day-28-28))][fe], df[(df['d'] >= 1000) & (df['d'] <= (day-28-28))]['sales'] 
# 1941-28-28到1941-28 作为测试集
vl_x, vl_y = df[(df['d'] > (day-28-28)) & (df['d'] <= (day-28))][fe], df[(df['d'] > (day-28-28)) & (df['d'] <= (day-28))]['sales']

train_data = lgb.Dataset(tr_x, label=tr_y)
valid_data = lgb.Dataset(vl_x, label=vl_y)

# 1914到1941 作为测试集，用训练的模型预测它的"sales"，再与真实的"sales"计算RMSE
test_df = df[df['d']>(day-28)].reset_index(drop=True)

# 定义黑盒函数
def lgb_cv(tweedie_variance_power, subsample, subsample_freq, learning_rate, num_leaves, min_data_in_leaf,
      feature_fraction, max_bin, n_estimators, lambda_l2, sub_row, sub_feature, bagging_freq):


    lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': tweedie_variance_power,
                    'metric': 'rmse',
                    'subsample': subsample,
                    'subsample_freq': int(subsample_freq),
                    'learning_rate': learning_rate,
                    'num_leaves': int(num_leaves),
                    'min_data_in_leaf': int(min_data_in_leaf),
                    'feature_fraction': feature_fraction,
                    'max_bin': int(max_bin),
                    'n_estimators': int(n_estimators),
                    'boost_from_average': False,
                    'seed': 42,  
                    'lambda_l2':lambda_l2,
                    'sub_row':sub_row,  
                    'sub_feature':sub_feature,
                    'bagging_freq':int(bagging_freq),               
                }

    stimator = lgb.train(lgb_params, train_data) 
    # valid_sets = [train_data, valid_data] 这个是用来显示training's rmse和valid_1's rmse，而 verbose_eval : 迭代多少次打印

    test_df['preds'] = stimator.predict(test_df[fe])
    base_score = rmse(test_df['sales'], test_df['preds'])

    return -base_score

# 给定超参数搜索空间
opt = BayesianOptimization(
                lgb_cv,
                {
                    'tweedie_variance_power': (1, 1.5),
                    'subsample': (0.5, 1.0),
                    'subsample_freq': (0, 5),
                    'learning_rate': (0, 1),
                    'num_leaves':(2**10-1,2**15-1),
                    'min_data_in_leaf':(2**10-1,2**15-1),
                    'feature_fraction':(0.4,1),
                    'max_bin':(80,150),
                    'n_estimators':(1000,1700),
                    'lambda_l2':(0,0.2),
                    'sub_row':(0.6,1),
                    'sub_feature':(0.6,1.0),
                    'bagging_freq':(0,5),
                }
              )
opt.maximize(n_iter = 20) # 最大化黑盒函数，迭代100次

rf_bo.max # 返回黑盒函数值最大的超参数

|   iter    |  target   | baggin... | featur... | lambda_l2 | learni... |  max_bin  | min_da... | n_esti... | num_le... | sub_fe... |  sub_row  | subsample | subsam... | tweedi... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -2.005    |  0.4722   |  0.4987   |  0.0196   |  0.03115  |  87.66    |  6.255e+0 |  1.438e+0 |  7.805e+0 |  0.7139   |  0.7916   |  0.5068   |  0.9146   |  1.496    |
|  2        | -2.31     |  4.875    |  0.4995   |  0.138    |  0.4799   |  145.5    |  4.86e+03 |  1.511e+0 |  1.923e+0 |  0.7036   |  0.8444   |  0.776    |  1.481    |  1.052    |
|  3        | -2.142    |  0.554    |  0.4238   |  0.09402  |  0.8591   |  118.5    |  2.158e+0 |  1.074e+0 |  2.461e+0 |  0.6854   |  0.68     |  0.5021   |  4.48     |  1.46     |
|  4        | -2.125    |  2.975    |  0.5173   |  0.1388   |  0.1773   |  95.7     |  3.5